# Анализ тональности с pytorch и RNN

[Этот ноутбук в colab](https://colab.research.google.com/drive/1VWAERpPsozIc2hVQ5WdA_ilI0tDQ4di5)

[источник](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb)

## Рекуррентная нейросеть

Проходится по последовательности (например, слов)  и на каждом шаге получает вектор слова и вектор предыдущего состояния.

$$h_t = \text{RNN}(x_t, h_{t-1})$$

![rnn.png](https://miro.medium.com/max/627/1*go8PHsPNbbV6qRiwpUQ5BQ.png)


## Данные

Мы будем использовать объекты класса `Field`. Они определяют, как данные будут храниться и обрабатываться.

В поле `TEXT` задаём `tokenize='spacy'`. Это определяет, что тексты будут токенизироваться с помошью [spaCy](https://spacy.io) tokenizer. Если не подать аргументов, тексты будут разбиваться по пробелам.

`LABEL` is defined by a `LabelField`, a special subset of the `Field` class specifically used for handling labels. We will explain the `dtype` argument later.

Больше про класс `Field` [здесь](https://github.com/pytorch/text/blob/master/torchtext/data/field.py).

In [0]:
import torch
from torchtext import data

SEED = 1234 # фиксируем seed для воспроизводимости

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

В pytorch (`torchtext.datasets`) хранятся некоторые стандартные датасеты. А ещё в них встроено разделение на train и test.

In [0]:
from torchtext import datasets

In [3]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.7MB/s]


Посмотрим сколько примеров в каждой части:

In [4]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


Посмотрим на один пример:

In [4]:
print(vars(train_data.examples[0]))

{'text': ['By', 'submitting', 'this', 'comment', 'you', 'are', 'agreeing', 'to', 'the', 'terms', 'laid', 'out', 'in', 'our', 'Copyright', 'Statement', '.', 'Your', 'submission', 'must', 'be', 'your', 'own', 'original', 'work', '.', 'Your', 'comments', 'will', 'normally', 'be', 'posted', 'on', 'the', 'site', 'within', '2', '-', '3', 'business', 'days', '.', 'Comments', 'that', 'do', 'not', 'meet', 'the', 'guidelines', 'will', 'not', 'be', 'posted', '.', 'Please', 'write', 'in', 'English', 'only', '.', 'HTML', 'or', 'boards', 'mark', '-', 'up', 'is', 'not', 'supported', 'though', 'paragraph', 'breaks', 'will', 'be', 'inserted', 'if', 'you', 'leave', 'a', 'blank', 'line', 'between', 'paragraph', '.', 'We', 'sent', 'an', 'e', '-', 'mail', 'to', 'when', 'you', 'registered', '.', 'You', 'must', 'click', 'on', 'the', 'link', 'in', 'that', 'e', '-', 'mail', 'to', 'complete', 'your', 'registration', 'and', 'enjoy', 'the', 'full', 'benefits', 'of', 'being', 'registered', 'at', 'IMDb.com', '.', '

In [5]:
train_data.examples[0].text[:10]

['By',
 'submitting',
 'this',
 'comment',
 'you',
 'are',
 'agreeing',
 'to',
 'the',
 'terms']

In [6]:
train_data.examples[0].label

'pos'

Создадим валидационную выборку с помощью метода `.split()`. Соотношение количества примеров можно задать с помощью аргумента `split_ratio`.

In [0]:
import random

In [0]:
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [9]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


Теперь построим _vocabulary_. Это, по большому счёту, таблица, в которой каждое слово соответствует индексу. Индекс используется для создания _one-hot_ векторов.

![](https://raw.githubusercontent.com/bentrevett/pytorch-sentiment-analysis/9210842371c3bbde7b2007051dafa4c74d9768cd/assets/sentiment5.png)

Количество уникальных слов 100,000 -- это очень много для векторов. Столько данных может не влезть в GPU. Поэтому мы возьмём 25,000 самых частых слов. Если нам встретится слово, которого среди них нет, оно заменится на  `<unk>`. Например, если в предожении "This film is great and I love it" не войдёт слово "love", получится "This film is great and I `<unk>` it".

In [0]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

Немного посмотрим на данные.

In [11]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


25002, а не 25000 -- из-за `<unk>` и `<pad>` токенов.

В батче должны быть однаковые длины предложений, если какое-то короче -- добавляем паддинг.

![](https://raw.githubusercontent.com/bentrevett/pytorch-sentiment-analysis/9210842371c3bbde7b2007051dafa4c74d9768cd/assets/sentiment6.png)

In [12]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 203532), (',', 193722), ('.', 165186), ('and', 109834), ('a', 109790), ('of', 100904), ('to', 93907), ('is', 76150), ('in', 61595), ('I', 54213), ('it', 53673), ('that', 49371), ('"', 44943), ("'s", 43516), ('this', 42153), ('-', 37005), ('/><br', 35751), ('was', 34927), ('as', 30584), ('with', 30068)]


Данные в словаре: `stoi` (**s**tring **to** **i**nt) и `itos` (**i**nt **to**  **s**tring).

In [13]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [14]:
print(TEXT.vocab.stoi['and'])

5


Следующий шаг подготовки данных -- создать итераторы по батчам.

Мы используем `BucketIterator`, который будет минимизировать добавлемые токены `<pad>`. А также мы переведём данные на GPU с помощью `torch.device`.

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

## Определяем модель

У нас три слоя:
* слой _embedding_ -- transform our sparse one-hot vector (sparse as most of the elements are 0) into a dense embedding vector (dense as the dimensionality is a lot smaller and all the elements are real numbers). Это полносвязный слой.
* RNN (takes in our dense vector and the previous hidden state $h_{t-1}$, which it uses to calculate the next hidden state, $h_t$.)
* слой _linear_ (transforming it to the correct output dimension)

![RNN](https://raw.githubusercontent.com/bentrevett/pytorch-sentiment-analysis/9210842371c3bbde7b2007051dafa4c74d9768cd/assets/sentiment7.png)

Теперь о том, что происходит в `forward`:
`text` -- это батч, тензор размера _**[sentence length, batch size]**_. Это набор предложений одинаковой длины. Вы могли заметить, что должна быть ещё одна размерность -- размер one-hot векторов для слов, но torch оченб удобно хранит их в виде индексов.

После слоя `Embedding` вектора слов становятся сжатыми, и данные приобретают размер _**[sentence length, batch size, embedding dim]**_.

`embedded` идёт в RNN. Вектор состояния на первом шаге автоматически инициализируется нулями.

RNN возвращает 2 тензора, `output` размера _**[sentence length, batch size, hidden dim]**_ и `hidden` размера _**[1, batch size, hidden dim]**_. `output` -- это скрытые состояния после каждого шага, а `hidden` -- только последнее. Метод `squeeze` убирает лишнее измерение. 

Наконец прогоняем данные через последний слой.

In [0]:
import torch.nn as nn

In [0]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim) # слой, которым мы представляем слова в виде векторов 

        self.rnn = nn.RNN(embedding_dim, hidden_dim) # клетка RNN
        
        self.fc = nn.Linear(hidden_dim, output_dim) # ещё один слой в конце
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [0]:
INPUT_DIM = len(TEXT.vocab) # размер one-hot векторов
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

Посчитаем параметры:

In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


## Обучаем модель

In [0]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

Определяем функцию потерь: _binary cross entropy with logits_. 

In [0]:
criterion = nn.BCEWithLogitsLoss()

Используем `.to` чтобы перенести вычисления на GPU (если она есть). 

In [0]:
model = model.to(device)
criterion = criterion.to(device)

Функция для подсчёта accuracy:

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

The `train` function iterates over all examples, one batch at a time. 

`model.train()` is used to put the model in "training mode", which turns on _dropout_ and _batch normalization_. Although we aren't using them in this model, it's good practice to include it.

For each batch, we first zero the gradients. Each parameter in a model has a `grad` attribute which stores the gradient calculated by the `criterion`. PyTorch does not automatically remove (or "zero") the gradients calculated from the last gradient calculation, so they must be manually zeroed.

We then feed the batch of sentences, `batch.text`, into the model. Note, you do not need to do `model.forward(batch.text)`, simply calling the model works. The `squeeze` is needed as the predictions are initially size _**[batch size, 1]**_, and we need to remove the dimension of size 1 as PyTorch expects the predictions input to our criterion function to be of size _**[batch size]**_.

The loss and accuracy are then calculated using our predictions and the labels, `batch.label`, with the loss being averaged over all examples in the batch.

We calculate the gradient of each parameter with `loss.backward()`, and then update the parameters using the gradients and optimizer algorithm with `optimizer.step()`.

The loss and accuracy is accumulated across the epoch, the `.item()` method is used to extract a scalar from a tensor which only contains a single value.

Finally, we return the loss and accuracy, averaged across the epoch. The `len` of an iterator is the number of batches in the iterator.

You may recall when initializing the `LABEL` field, we set `dtype=torch.float`. This is because TorchText sets tensors to be `LongTensor`s by default, however our criterion expects both inputs to be `FloatTensor`s. Setting the `dtype` to be `torch.float`, did this for us. The alternative method of doing this would be to do the conversion inside the `train` function by passing `batch.label.float()` instad of `batch.label` to the criterion. 

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train() 
    
    for batch in iterator:
        optimizer.zero_grad() # обнуляем градиенты

        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)

        loss.backward() # считаем градиенты
        optimizer.step() # обновляем переметры
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

`evaluate` is similar to `train`, with a few modifications as you don't want to update the parameters when evaluating.

`model.eval()` puts the model in "evaluation mode", this turns off _dropout_ and _batch normalization_. Again, we are not using them in this model, but it is good practice to include them.

No gradients are calculated on PyTorch operations inside the `with no_grad()` block. This causes less memory to be used and speeds up computation.

The rest of the function is the same as `train`, with the removal of `optimizer.zero_grad()`, `loss.backward()` and `optimizer.step()`, as we do not update the model's parameters when evaluating.

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Функция для оценки времени обучения:

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Обучаем несколько эпох, на каждой эпохе оцениваем качество.

In [69]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 45s
	Train Loss: 0.693 | Train Acc: 50.18%
	 Val. Loss: 0.694 |  Val. Acc: 48.81%
Epoch: 02 | Epoch Time: 0m 45s
	Train Loss: 0.693 | Train Acc: 49.91%
	 Val. Loss: 0.694 |  Val. Acc: 48.83%
Epoch: 03 | Epoch Time: 0m 44s
	Train Loss: 0.693 | Train Acc: 50.14%
	 Val. Loss: 0.694 |  Val. Acc: 48.91%
Epoch: 04 | Epoch Time: 0m 45s
	Train Loss: 0.693 | Train Acc: 50.15%
	 Val. Loss: 0.694 |  Val. Acc: 48.94%
Epoch: 05 | Epoch Time: 0m 44s
	Train Loss: 0.693 | Train Acc: 50.54%
	 Val. Loss: 0.694 |  Val. Acc: 48.87%


Loss уменьшается слабовато. Потому что эта модель далека от идеала. 

Теперь измерим главную метрику -- accuracy на тесте.


In [70]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.700 | Test Acc: 40.63%


## Задания

* Измените архитектуру нейросети так, чтобы перед последним слоем был dropout (`nn.Dropout`); измерьте качество
* попробуйте другой оптимизатор, например `optim.Adam`
* специально обработаем токены `<pad>`